In [3]:
import ipyparallel as ipp
from ipyparallel.util import interactive

In [5]:
rc = ipp.Client(profile='myprofile', sshserver='bear@brahms')

255
b'ssh: connect to host brahms port 22: Connection refused\r\r\n'
<class 'pexpect.exceptions.EOF'>


RuntimeError: tunnel 'ssh  -f -S none -L 127.0.0.1:58185:127.0.0.1:38229 bear@brahms sleep 10' failed to start

In [ ]:
rc.ids

In [ ]:
dview = rc[:]

In [ ]:
with dview.sync_imports():
    import numpy as np
    import matplotlib.pyplot as plt
    import pickle

In [ ]:
from EM import EM
from plot import plot

In [ ]:
signal_context = pickle.load(open('../Data/signal_context.pkl', 'rb'))

In [ ]:
# EM Setting
training_pct = .8
single_effect = False

In [ ]:
all_signals = list(signal_context.keys())

In [ ]:
# run em on a given signal
# return em object after convergence
def run_em_on_signal(signal_name):
    import sys
    sys.path.insert(0, '/home/bear/Documents/Counterfactual-Prediction/EM.py')
    from EM import EM
    if signal_name == 'inr':
        num_past_effects = 3
    else:
        num_past_effects = 2
    #treatment_names, chronic_keywords, demographic_names = signal_context[signal_name]
    data = np.load('~/Document/Data/'+signal_name+'_preprocessed_data.npz')
    em = EM(data['y_mtx'], data['X_mtx'], data['c_mtx'], num_past_effects, K=0, train_pct=training_pct, X_prev_given=True,
        X_prev = data['X_prev_mtx'], single_effect=single_effect)
    em.run_EM(1)
    return em

In [ ]:
result = v.map_sync(run_em_on_signal, all_signals)

In [ ]:
result.get()

In [ ]:
%%time
past_effects = [2]
all_params = {}

for s, signal_name in enumerate(all_signals):
    num_past_effects = past_effects[s]
    treatment_names, chronic_keywords, demographic_names = signal_context[signal_name]
    data = np.load('../Data/'+signal_name+'_preprocessed_data.npz')
    
    num_iter = 1 # number of iterations to average over coefficients
    # store coefficients averaged over multiple runs of EM
    A = np.zeros((num_iter, num_past_effects, len(treatment_names.keys())))
    b = np.zeros((num_iter, len(chronic_keywords) + len(demographic_names)))
    transition_std = np.zeros(num_iter)
    observation_std = np.zeros(num_iter)
    initial_state_std = np.zeros(num_iter)
    initial_state_mean = np.zeros(num_iter)
    mse = []
    loglik = []
    intercept = []

    for i in range(num_iter):
        em = EM(data['y_mtx'], data['X_mtx'], data['c_mtx'], num_past_effects, K=0, train_pct=training_pct, X_prev_given=True,
            X_prev = data['X_prev_mtx'], single_effect=single_effect)
        em.run_EM(2000)
        A[i, :, :] = em.A
        b[i, :] = em.b
        initial_state_mean[i] =  em.init_z
        initial_state_std[i] = np.sqrt(em.sigma_0)
        transition_std[i] = np.sqrt(em.sigma_1)
        observation_std[i] = np.sqrt(em.sigma_2)
        mse.append(em.mse)
        loglik.append(em.obs_log_lik)
        intercept.append(em.intercept)
        print('Finish run {} for {}'.format(i+1, signal_name))
        
    all_params[signal_name] = [A, b, initial_state_mean, initial_state_std, transition_std, observation_std, mse, loglik, intercept]

#pickle.dump(all_params, open('../Data/est_params.pkl', 'wb'))

In [ ]:
#all_params = pickle.load(open('../Data/est_params.pkl', 'rb'))
for signal_name in all_signals:
    learned_params = all_params[signal_name]
    A = learned_params[0]
    b = learned_params[1]
    initial_state_mean = learned_params[2]
    initial_state_std = learned_params[3]
    transition_std = learned_params[4]
    observation_std = learned_params[5]
    #intercept = learned_params[8]
    treatment_names, chronic_keywords, demographic_names = signal_context[signal_name]
    print('{} coefficients averaged over {} runs'.format(signal_name, num_iter))
    for i, treatment in enumerate(treatment_names.keys()):
        print('Coefficient for {}: {}'.format(treatment, np.round(np.average(A, axis=0)[:, i], 4)))
    static_types = chronic_keywords + demographic_names
    for j, static in enumerate(static_types):
        print('Coefficient for {}: {}'.format(static, np.round(np.average(b, axis=0)[j], 4)))
    print('initial state mean: {}'.format(np.round(np.average(initial_state_mean), 4)))
    print('initial state std: {}'.format(np.round(np.average(initial_state_std), 4)))
    print('transition noise (std): {}'.format(np.round(np.average(transition_std), 4)))
    print('observation noise (std): {}'.format(np.round(np.average(observation_std), 4)))
    #print('intercept: {}'.format(np.round(np.average(intercept), 8)))
    print('')

In [ ]:
for signal_name in all_signals:
    loglik = all_params[signal_name][7][0]
    plt.plot([i for i in range(len(loglik))], loglik)
    plt.xlabel('Iterations')
    plt.ylabel('Observed data log likelihood')
    plt.title('EM log likelihood of {}'.format(signal_name))
    plt.show()

In [ ]:
for signal_name in all_signals:
    mse = all_params[signal_name][6][0]
    plt.plot([i for i in range(len(mse))], mse)
    plt.xlabel('Iterations')
    plt.ylabel('MSE')
    plt.title('Prediction MSE of {}'.format(signal_name))
    plt.show()

In [ ]:
for signal_name in all_signals:
    learned_params = all_params[signal_name]
    A = learned_params[0]
    b = learned_params[1]
    initial_state_mean = learned_params[2]
    initial_state_std = learned_params[3]
    transition_std = learned_params[4]
    observation_std = learned_params[5]
    treatment_names, chronic_keywords, demographic_names = signal_context[signal_name]
    print('{} coeffients std'.format(signal_name))
    for i, treatment in enumerate(treatment_names.keys()):
        print('Coefficient for {}: {}'.format(treatment, np.round(np.std(A, axis=0)[:, i], 4)))
    static_types = chronic_keywords + demographic_names
    for j, static in enumerate(static_types):
        print('Coefficient for {}: {}'.format(static, np.round(np.std(b, axis=0)[j], 4)))
    print('initial state mean: {}'.format(np.round(np.std(initial_state_mean), 4)))
    print('initial state std: {}'.format(np.round(np.std(initial_state_std), 4)))
    print('transition noise (std): {}'.format(np.round(np.std(transition_std), 4)))
    print('observation noise (std): {}'.format(np.round(np.std(observation_std), 4)))
    print('')

In [ ]:
plt.plot([i for i in range(len(em.obs_log_lik))], em.obs_log_lik)
plt.xlabel('Iterations')
plt.ylabel('MSE')
plt.title('Prediction MSE of {}'.format(signal_name))
plt.show()

In [ ]:
for i, treatment in enumerate(treatment_names.keys()):
    print('Coefficient for {}: {}'.format(treatment, np.round(em.A[:, i], 4)))
static_types = chronic_keywords + demographic_names
for j, static in enumerate(static_types):
    print('Coefficient for {}: {}'.format(static, np.round(em.b[j], 4)))
print('initial state mean: {}'.format(round(em.init_z, 4)))
print('initial state std: {}'.format(round(np.sqrt(em.sigma_0), 4)))
print('transition noise (std): {}'.format(round(np.sqrt(em.sigma_1), 4)))
print('observation noise (std): {}'.format(round(np.sqrt(em.sigma_2), 4)))
print('intercept: {}'.format(em.intercept))

In [ ]:
patient_index = 0
y_mtx = data['y_mtx'][patient_index, :]
X_mtx = data['X_mtx'][patient_index, :, :]
c_mtx = data['c_mtx'][patient_index, :]
X_prev_mtx = data['X_prev_mtx'][patient_index, :, :]
y_mtx = y_mtx[np.newaxis, :]
X_mtx = X_mtx[np.newaxis, :, :]
c_mtx = c_mtx[np.newaxis, :]
X_prev_mtx = X_prev_mtx[np.newaxis, :, :]

In [ ]:
converge = np.array(em.sos)
initial = np.array(a)

In [ ]:
diff = converge - initial

In [ ]:
np.argsort(diff)[-30:]

In [ ]:
random_patients = np.random.randint(0, em.y.shape[0], 20)
for i in random_patients:
    plot(em, i, 12, signal_name, treatment_types=list(treatment_names.keys()))

### Analysis
Changes
* Only select patients with at least 4 training observations (total of 1821)

On real data
* Issues:
    * treatments coefficients are negative while they should be positive
        * changing initialization doesn't seem to help as model converges in bad local min, and most directions still don't change
    * observation noise is bigger than transition noise
    * run time is long, each iteration takes 4s, M step takes 2.5s, loglik takes 1s
* Reasons:
    * The increasing effects of treatments are not pronouced in the data (see Source Data Analysis)
        * it makes sense that nsaid and aspirin shows small number close to zero since their effects are 50-50.
        * transfusion platelet should be more negative than others, also makes sense.
        * the first number for plasma is a very negative number, but should be close to zero
        * anticoagulant are more negative than platelet, but should be more similar
    * bad noise estimation is expected, specifically underestimated initial state variance and overestimated observational noise 
    
On simulation
* Issues
    * b seems to be harder to identify correctly compared to A (sensitive to initialization)
    * estimated sigma 0 and sigma 2 are very different from the true value (not sensitive to initialization)
        * underestimate sigma 0, overestimate sigma 2
* Reasons
   